In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\Shlok Mathur\Desktop\practice_codes\Planets\planets_updated.csv")
df.head(1)

,Planet,Color,Mass (10^24kg),Diameter (km),Density (kg/m^3),Surface Gravity(m/s^2),Escape Velocity (km/s),Rotation Period (hours),Length of Day (hours),Distance from Sun (10^6 km),...,Mean Temperature (C),Surface Pressure (bars),Number of Moons,Ring System?,Global Magnetic Field?,Surface Temperature (C),Atmospheric Composition,Atmospheric Pressure (bars),Surface Features,Composition
0,Mercury,Grey,0.33,4879,5429,3.7,4.3,1407.6,4222.6,57.9,...,167,0,0,No,Yes,-173 to 427,Mostly None,Trace,"Craters, Scarps",Rock and Metal


In [3]:
orbital_days = [88, 225, 365.25, 687, 4333, 10759, 30687, 60190]  # Correct number of values (8 planets)

df['Orbital_Days'] = orbital_days


df.to_csv(r'C:\Users\Shlok Mathur\Desktop\practice_codes\Planets\planets_updated_with_days.csv', index=False)



In [4]:
df.columns

Index(['Planet', 'Color', 'Mass (10^24kg)', 'Diameter (km)',
       'Density (kg/m^3)', 'Surface Gravity(m/s^2)', 'Escape Velocity (km/s)',
       'Rotation Period (hours)', 'Length of Day (hours)',
       'Distance from Sun (10^6 km)', 'Perihelion (10^6 km)',
       'Aphelion (10^6 km)', 'Orbital Period (days)',
       'Orbital Velocity (km/s)', 'Orbital Inclination (degrees)',
       'Orbital Eccentricity', 'Obliquity to Orbit (degrees)',
       'Mean Temperature (C)', 'Surface Pressure (bars)', 'Number of Moons',
       'Ring System?', 'Global Magnetic Field?', 'Surface Temperature (C)',
       'Atmospheric Composition', 'Atmospheric Pressure (bars)',
       'Surface Features', 'Composition', 'Orbital_Days'],
      dtype='object')

In [5]:

import numpy as np

file_path = r"C:\Users\Shlok Mathur\Desktop\practice_codes\Planets\planets_updated_with_days.csv"
df = pd.read_csv(file_path)

df['Semi-major axis (a) (10^6 km)'] = (df['Perihelion (10^6 km)'] + df['Aphelion (10^6 km)']) / 2
df['Semi-minor axis (b) (10^6 km)'] = np.sqrt(df['Perihelion (10^6 km)'] * df['Aphelion (10^6 km)'])


df['Perimeter (10^6)(km)'] = np.pi * (3 * (df['Semi-major axis (a) (10^6 km)'] 
+ df['Semi-minor axis (b) (10^6 km)']) - np.sqrt((3 * df['Semi-major axis (a) (10^6 km)']
 + df['Semi-minor axis (b) (10^6 km)']) *
(df['Semi-major axis (a) (10^6 km)'] + 3 * df['Semi-minor axis (b) (10^6 km)'])))


columns_to_load = [
    'Planet',  
    'Diameter (km)',
    'Rotation Period (hours)',
    'Length of Day (hours)',
    'Distance from Sun (10^6 km)',
    'Perihelion (10^6 km)',
    'Aphelion (10^6 km)',
    'Orbital Period (days)',
    'Orbital Velocity (km/s)',
    'Orbital Inclination (degrees)',
    'Orbital_Days',
    'Perimeter (10^6)(km)'
]

df = df[columns_to_load]


output_file_path = r"C:\Users\Shlok Mathur\Desktop\practice_codes\modified_planets.csv"
df.to_csv(output_file_path, index=False)

output_file_path  


'C:\\Users\\Shlok Mathur\\Desktop\\practice_codes\\modified_planets.csv'

In [6]:
fd = pd.read_csv(r"C:\Users\Shlok Mathur\Desktop\practice_codes\Planets\modified_planets.csv")
fd.columns

Index(['Planet', 'Diameter (km)', 'Rotation Period (hours)',
       'Length of Day (hours)', 'Distance from Sun (10^6 km)',
       'Perihelion (10^6 km)', 'Aphelion (10^6 km)', 'Orbital Period (days)',
       'Orbital Velocity (km/s)', 'Orbital Inclination (degrees)',
       'Orbital_Days', 'Perimeter (10^6)(km)'],
      dtype='object')

In [7]:


from datetime import datetime

planet_info = pd.read_csv(r'C:\Users\Shlok Mathur\Desktop\practice_codes\Planets\modified_planets.csv')  # Updated columns
coordinates = pd.read_csv(r'C:\Users\Shlok Mathur\Desktop\practice_codes\Planets\planetary_coordinates_2025_01_16.csv')  # Updated columns

def calculate_angle(revolution_days, velocity, perimeter, time_difference):
    
    distance_travelled = velocity * time_difference  
    angle_moved = (distance_travelled / perimeter) * 360 
    return angle_moved


def get_new_coordinates(user_date_str):
  
    user_date = datetime.strptime(user_date_str, "%Y-%m-%d") 
    date_16jan2025 = datetime(2025, 1, 16)
    
   
    time_difference = (user_date - date_16jan2025).days

   
    new_coordinates = []
    for _, planet_row in planet_info.iterrows():
        planet = planet_row['Planet']
        revolution_days = planet_row['Orbital Period (days)']
        velocity = planet_row['Orbital Velocity (km/s)'] * 86400 
        perimeter = planet_row['Perimeter (10^6)(km)'] * 1e6 
        inclination = np.radians(planet_row['Orbital Inclination (degrees)']) 

        
        initial_coords = coordinates[coordinates['Planet'] == planet].iloc[0]
        x, y, z = initial_coords['X (AU)'], initial_coords['Y (AU)'], initial_coords['Z (AU)']
        
        
        angle_moved = calculate_angle(revolution_days, velocity, perimeter, time_difference)

       
        angle_radians = np.radians(angle_moved)

        new_x = x * np.cos(angle_radians) - y * np.sin(angle_radians)
        new_y = x * np.sin(angle_radians) + y * np.cos(angle_radians)
       
        new_z = z * np.cos(inclination) 
        
      
        new_coordinates.append({
            'Planet': planet,
            'New_X (AU)': new_x,
            'New_Y (AU)': new_y,
            'New_Z (AU)': new_z
        })

    
    new_coords_df = pd.DataFrame(new_coordinates)
    return new_coords_df


user_date_str = input("Enter the new date (YYYY-MM-DD): ") 
new_coords = get_new_coordinates(user_date_str)


output_filename = r"C:\Users\Shlok Mathur\Desktop\practice_codes\Planets\planet_coordinates_new.csv"
new_coords.to_csv(output_filename, index=False)


print(f"New Coordinates of the Planets on {user_date_str} have been saved to '{output_filename}'")


New Coordinates of the Planets on 2020-12-12 have been saved to 'C:\Users\Shlok Mathur\Desktop\practice_codes\Planets\planet_coordinates_new.csv'
